In [1]:
import os
import random

import haven.db as db

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [2]:
tag_keys = sorted(list(read_data_w_cache('select distinct tag_key from mgietzmann_tag_tracks')['tag_key']))

In [3]:
len(tag_keys)

111

In [10]:
random.seed(42)
test_cases = random.sample(tag_keys, 20)
train_cases = [k for k in tag_keys if k not in test_cases]
validation_cases = random.sample(train_cases, 20)
train_cases = [k for k in train_cases if k not in validation_cases]
print(len(test_cases), len(validation_cases), len(train_cases))

20 20 71


In [11]:
test_cases

['210770',
 '159006',
 '142191',
 '229209',
 '172908',
 '172904',
 '159020',
 '159008b',
 '159005b',
 '210775',
 '205413',
 '159002b',
 '210763',
 '202599',
 '142194',
 '159019',
 '172901',
 '205406',
 '210765',
 '205417']

In [12]:
data = read_data_w_cache(
    'select * from mgietzmann_tag_tracks'
)
data.head()

,tag_key,epoch,longitude,latitude,upload_key
0,129843,1387411200,-166.922615,54.131760,mgietzmann
1,129843,1387497600,-166.884086,54.258072,mgietzmann
2,129843,1387584000,-166.910525,54.312433,mgietzmann
3,129843,1387670400,-166.817057,54.358280,mgietzmann
4,129843,1387756800,-166.676901,54.389694,mgietzmann


In [13]:
data['_train'] = False 
data.loc[data['tag_key'].isin(train_cases), '_train'] = True
data.groupby('_train').size()

_train
False    2639
True     4893
dtype: int64

In [16]:
db.write_data(
    data[data['tag_key'].isin(test_cases)], 'mgietzmann_tag_tracks_test', ['upload_key']
)
db.write_data(
    data[~data['tag_key'].isin(test_cases)], 'mgietzmann_tag_tracks_train', ['upload_key']
)